Visualization of tremor catalog with bokeh

Import Python packages

In [1]:
from bokeh.io import curdoc
from bokeh.layouts import column, gridplot, layout, widgetbox
from bokeh.models import BoxZoomTool, ColumnDataSource, PanTool, ResetTool, WheelZoomTool
from bokeh.models.widgets import DateSlider
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON

from os.path import join, dirname
import pandas as pd

Define functions

In [2]:
def make_plot(selected_data_source, unselected_data_source):    
    tools = [ResetTool(), PanTool(), BoxZoomTool(), WheelZoomTool()]
    
    xmin, xmax = tremors.merc_long.min(), tremors.merc_long.max()
    ymin, ymax = tremors.merc_lat.min(), tremors.merc_lat.max()

    top = figure(tools=tools, plot_width=600, plot_height=500, title='Tremor locations', x_range=(xmin, xmax), y_range=(ymin, ymax),
               x_axis_type="mercator", y_axis_type="mercator")
    top.add_tile(CARTODBPOSITRON)
    top.circle('merc_long', 'merc_lat', source=unselected_data_source, color='gray', alpha=0.2)
    top.circle('merc_long', 'merc_lat', source=selected_data_source, color='red', alpha=1.0)#, view=view)

    bottom = figure(plot_width=600, plot_height=80, title=None, x_axis_type="datetime")
    bottom.line('date', 'value', source=ColumnDataSource(tremors), color='red', alpha=0.8)

    p = gridplot([[top], [bottom]], toolbar_location='right')
    
    return p

In [3]:
def get_datasets(tremors):
    '''Filter based on widget
    '''
    # start with the whole dataset
    copy = tremors.copy()
    # narrow by selected date from the DateSlider
    selected = copy[copy['datetime']==pd.Timestamp(date_slider.value)]
    unselected = copy[copy['datetime'] < pd.Timestamp(date_slider.value)]

    return {'selected':ColumnDataSource(data=selected), 'unselected':ColumnDataSource(unselected)}

In [4]:
def update_plot():
    src = get_datasets(df)
    selected_data_source.data.update(src['selected'].data)
    unselected_data_source.data.update(src['unselected'].data)

Read tremor data

In [5]:
filename = '../data/tremors.pkl'
tremors = pd.read_pickle(filename)

Keep only 10 per cent of the tremor (to get a smaller dataset)

In [6]:
tremors = tremors.sample(frac=0.1)

Lauch code

In [7]:
date_slider = DateSlider(start=tremors['datetime'].min(), end=tremors['datetime'].max(), step=100, value=tremors['datetime'].min())
date_slider.on_change('value', lambda attr, old, new: update_plot())

slider_box = widgetbox(children=[date_slider], width=600)

datasets_init = get_datasets(tremors) # get bokeh ColumnDataSource
selected_data_source = datasets_init['selected']
unselected_data_source = datasets_init['unselected']
p = make_plot(selected_data_source, unselected_data_source)

layout = layout(children=[[p],[slider_box]], sizing_mode='fixed')

curdoc().add_root(layout)